# Assignment 8

Name 1: <br/>
Student id 1: <br/>
Email 1: <br/>


Name 2: <br/>
Student id 2: <br/>
Email 2: <br/> 

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the Python files for exercise 1. There is no need to submit the data files. <br/>
Upload the zipped folder in Teams. Make sure to click on "Turn-in" after your upload your submission, otherwise the assignment will not be considered as submitted. Only one from the group should make the submisssion.

---

## Exercise 1: Document Classification Warm-Up (4 points)

In this exercise you will explore the Reuters News corpus and apply basic feature selection techniques to it. You can download the reuters corpus and English stop words via [NLTK](https://www.nltk.org/):

In [ ]:
!pip install nltk
import nltk
nltk.download('reuters')
nltk.download('stopwords')

It consists of articles from [Reuters](https://de.wikipedia.org/wiki/Reuters) mapped to one of 
Files and categories (classes) can be accessed easily. Although there is a train/test split, we will use the whole corpus for this exercise.

In [ ]:
from nltk.corpus import reuters, stopwords
stop_words = stopwords.words('english')
print(reuters.categories())
print("# of categories: {}".format(len(reuters.categories())))
reuters.fileids()[:10]

Each document belongs to one or more categories. For the sake of simplicity we map each document to the first of its categories.

In [ ]:
print(reuters.categories('test/14833'))
category = reuters.categories('test/14833')[0]
category

**1.1: Categories (1.5 points)**

1. Load data into the `Corpus` class. We suggest to give an attribute `documents` that contains a list of `Document` objects. Preprocessing steps (tokenization, lowercasing, stopword removal) should happen within the `Document` objects. As always, you can deviate from that if you find it more convenient. (1 point)
2. Implement the method `category_frequencies` in the `Corpus` class. It should calculate the absolute frequencies of each category in the whole corpus. (0.25 points)
3. Plot categories vs. their frequency on a log-log scale. Do so by implementing the function `plot_category_frequencies`. What can you say about the distribution the category frequencies you observe? (0.25 points)


In [ ]:
from importlib import reload
import exercise_1
exercise_1 = reload(exercise_1)

corpus = exercise_1.Corpus(
    documents=[exercise_1.Document(fileid, reuters.raw(fileid), reuters.categories(fileid), stop_words=stop_words) for fileid in reuters.fileids()],
    categories=reuters.categories()
)

exercise_1.plot_category_frequencies(corpus.category_frequencies())

**1.2 Document Frequency (1.5 points)**

A term's document frequency (DF) is given by:

\begin{equation}
DF_{c_k}(t_i) = 
\frac{
\text{# of times $t_i$ occurs in $C_k$}
}{
\text{# of documents of category $C_k$}
}
\end{equation}

1. Find the 10 most frequent terms of the `housing ` category. Do so by implementing the method `term_frequencies` in the `Corpus` class. (0.5 points)
2. For these 10 terms, calculate their DF in the `housing` category as well as in the whole corpus (over all categories). Do so by implementing the method`df` in the `corpus` class. Plot the results by implementing the function `plot_dfs`. (0.5 points)
3. Is DF a good predictor for a category? Explain in 2-3 sentences. (0.5 points)

In [ ]:
most_common_housing = corpus.term_frequencies('housing').most_common(10)
most_common_housing = [term for term, _ in most_common_housing]

dfs_housing = [corpus.df(term, category='housing') for term in most_common_housing]
dfs_all = [corpus.df(term) for term in most_common_housing]

exercise_1.plot_dfs(most_common_housing, dfs_housing)
exercise_1.plot_dfs(most_common_housing, dfs_all)

**1.3 Pointwise Mutual Information (1 point)**

PMI is given by:
\begin{equation}
PMI(x;y) = \log_2 \frac{p(x,y)}{p(x)p(y)}
\end{equation}

1. Give an interpretation of PMI. When will it be large, when will it be small? What are its upper and lower bound? (0.25 points)
2. Calculate PMI of the `housing` category and each of the 10 most frequent terms from that category. Do so by implementing the method `pmi` in the `Corpus` class. Plot the PMI values. Do so by implementing the function `plot_pmis`. (0.5 points)
3. What does the plot tell you about the suitability of the most frequent words of a category as a feature for text classification? Come up with an improved measure that makes use of both the document frequency and PMI (it need not be derived formally from PMI, but it should be motivated by your findings). (0.25 points)

In [ ]:
pmis = [corpus.pmi('housing', term) for term in most_common_housing]
exercise_1.plot_pmis('housing', most_common_housing, pmis)

## Exercise 2 $\chi^2$ (2p)

Study the usage of $\chi^2$ statistics online. A very brief overview with examples can be found on [Wikipedia](https://en.wikipedia.org/wiki/Chi-squared_test).

**2.1 $\chi^2$ Introduction (1.25 points)**

1. How can you turn a numerical feature into a categorical one? What are the parameters of this decision? What are some features (at least 3) one may encounter in NLP? (0.25 points)
2. With the following knowledge of binary features, what's the expected count of $c(A \cap B), c(\neg A \cap B), c(A \cap \neg  B), c(\neg A \cap \neg B)$? \\
Total count $N = 100$, $c(A) = 35, c(B) = 61$. (0.25 points)
3. Using the given observations compute the $\chi^2$ statistics and the respective probability of the features $B$ and $C$. (0.5 points)

||A|$\neg$ A|
|-|-|-|
|B|6|55|
|$\neg$ B|29|10|

||A|$\neg$ A|
|-|-|-|
|C|21|40|
|$\neg$ C|14|25|

4. Based on your research of $\chi^2$, what does the computed probability signify (in terms of hypotheses)? What is the hypothesis? Would you say that $B$ and $C$ are good features to decribe $A$? (0.25 points)


**2.2 $\chi^2$ Feature selection (0.75 points)**

1. Given the set of categorical features $F$ and one specific target feature $f_0$, how can you use the $\chi^2$ statistics to select the most predictive feature? (0.25 points)
2. If you know all the features in $F$ are **not** pair-wise independent, how do you select $k$ features to describe $f_0$ together? (0.25 points)
3. If you know all the features in $F$ are pair-wise independent, how do you select $k$ features to describe $f_0$ together? (Try to be computationally efficient and therefore provide a different algorithm than for the previous question.) (0.25 points)

## Exercise 3: Authorship Identification (4 points)

The science of *stylometry* assumes that authors have unconscious writing habits. Therefore, the words and grammar in a text become reliable indicators of the author of the text. Various research techniques have tried to do a feature-level analysis for the task of authorship identification/attribution. 

**3.1 (1 point)** 

In Chapter 6, you studied different types of classification. For the particular problem of authorship identification, suggest how you could formulate this task for each of the following categories (1 sentence each)

- Binary classification
- Multi-class classification
- Flat classification
- Hierarchical classification
- Single category classification
- Multi-category classification
- Clustering

**3.2 (2.5 points)**

Read this [paper](https://dl.acm.org/doi/pdf/10.3115/1220355.1220443) which does a feature study for authorship identification. Answer the following questions (1-3 sentences each):

1. What are the 6 categories of features extracted for the task? Explain in 1 sentence each. (0.5 points)
2. What does the author observe in comparison to the baseline? (0.5 points) 
3. The author suggests using [SVM](https://en.wikipedia.org/wiki/Support-vector_machine)s as the classifier. What are the advantages offered by SVMs which guarantee a high accuracy in this problem? (0.5 point) 
4. The author concludes several possible scopes of improvement. Based on what you have studied so far, suggest if any improvements can be made in the given study w.r.t. (1 point)
  - Feature selection 
  - Usage of n-gram features
  - Any other approach other than the above two

**3.3 (0.5 points)** 

State 3 real-life, contemporary applications where authorship identification/verification/attribution is useful. 

## Bonus (1.5 points) 

Consider the two plots given below. 

* For Plot 1 and Plot 2, are any or both of the features 1 and 2 
1. redundant
2. useful
3. irrelevant

    for a task like clustering? Explain your choice from the above 3. (1 point)
    
![Plots](features.png "Plots 1 and 2")

* Draw/create a plot which shows features 1 and 2 as the remaining of the 3 choices stated above. Explain your reasoning in 1-2 sentences. (0.5 point)